In [1]:
import csv
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from verstack.stratified_continuous_split import scsplit # pip install verstack

# Load the training data
data_all = pd.read_csv("data/train.csv")

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import random

In [4]:
print(data_all.describe())
print(data_all.head())

                  id     timestamp  retweet_count  user_statuses_count  \
count  665777.000000  6.657770e+05  665777.000000         6.657770e+05   
mean   332888.000000  1.588500e+12     147.687398         4.167295e+04   
std    192193.409422  1.390942e+08    2972.051181         9.848516e+04   
min         0.000000  1.588284e+12       0.000000         0.000000e+00   
25%    166444.000000  1.588358e+12       0.000000         2.352000e+03   
50%    332888.000000  1.588513e+12       0.000000         1.080400e+04   
75%    499332.000000  1.588625e+12       2.000000         3.809900e+04   
max    665776.000000  1.588716e+12  942572.000000         7.203222e+06   

       user_followers_count  user_friends_count  
count          6.657770e+05        6.657770e+05  
mean           2.329881e+05        2.743131e+03  
std            2.442260e+06        1.725410e+04  
min            0.000000e+00        0.000000e+00  
25%            2.060000e+02        2.290000e+02  
50%            1.002000e+03      

In [ ]:
#covid corona, trump

In [7]:

modified = pd.read_csv("data/train.csv")
modified.user_verified = (modified.user_verified).astype(int)

modified.user_mentions = modified.user_mentions.fillna(0)
modified.user_mentions = modified.user_mentions.apply(lambda x: 1 if type(x) is str and x!=0 else 0) 

modified.urls = modified.urls.fillna(0)
modified.urls = modified.urls.apply(lambda x: 1 if type(x) is str and x!=0 else 0) 


#NLP stuff

modified.hashtags = modified.hashtags.fillna(0)
#modified.hashtags = modified.hashtags.apply(lambda x: 1 if type(x) is str and x!=0 else 0)

modified.text = modified.text.fillna(0)
#modified.text = modified.text.apply(lambda x: len(x) if type(x) is str and x!=0 else 0) 







In [8]:
print(modified.head())

   id      timestamp  retweet_count  user_verified  user_statuses_count  \
0   0  1588696955143              0              0                68460   
1   1  1588464948124              0              0                  309   
2   2  1588634673360              0              0                 3241   
3   3  1588433158672              0              0                32327   
4   4  1588582751599              0              0                  581   

   user_followers_count  user_friends_count  user_mentions  urls hashtags  \
0                  1101                1226              0     0        0   
1                    51                 202              0     0        0   
2                  1675                2325              0     0        0   
3                   667                 304              0     0        0   
4                    42                 127              0     0        0   

                                                text  
0                              

In [12]:
print(modified.hashtags[modified.hashtags!=0])

6                             Ethiopia
13          coronavirus, AnimalCruelty
16        COVID19, auspol, coronavirus
25                  SpaceForDistancing
26                  Coronavirus, Virus
                      ...             
665734                         COVID19
665745       COVID19, MentalHealthWeek
665757                  APFightsCorona
665765                       Singapore
665773                         COVID19
Name: hashtags, Length: 72451, dtype: object


In [26]:
non_empty_hashtags = modified.hashtags[modified.hashtags!=0]

unique_hashtags = set()

for a in non_empty_hashtags:
    b = a.split(', ')
    for c in b:
        unique_hashtags.add(c.lower())
        
    

In [27]:
print(len(unique_hashtags))
print(unique_hashtags)

29916
{'emplaw', 'girlmedtwitter', 'rss', 'learningfromhome', 'endmassincarceration', 'traumateam', 'acvc_esc', 'experiment', 'ministryofhealth', 'bolsonaroheroi', 'climatecrisis', 'day32oflockdown', 'bellanaijaweddjngs', 'yorkuscs', 'openforbusiness', 'celebratevaccines', 'taxheaven', 'kuwait', 'broadcasting', 'cipsac', 'coxbazar', 'pacificislander', 'france', 'iiar', 'it', 'givemom', 'nfid', 'pharmacyhour', 'measurement', 'pslhub', 'nonprofitmilkbanking', 'ppeshortage', 'gcc', 'jetstreamfest', 'uganig', 'orleg', 'adbnews', 'coop', 'sakagibi', 'wifi', 'asymmetric', 'nopanic', 'microsoft', 'bamboonow', 'jco', 'acscovid19', '繼續努力', 'staysafeonset', 'fidaupdate', 'faucihero', 'panickbuying', 'kalutara', 'provemewrong', 'fightcovid19withkaownah', 'traveling', 'day17oflockdown', 'margarita', '26aprile', 'tnfightscorona', 'increasedisabilitybenefits', 'avellinovittorio', 'nudging', 'pandemicproblems', 'covid19ebook', 'aaranyak', 'divine', 'cklw', 'cyblockhybrid', 'mla', 'vitamind', 'myfirst

In [28]:
non_empty_text = modified.text[modified.text!=0]

unique_text_hashtags = set()

for a in non_empty_text:
    a2 = a.replace('\n',' ')
    b = a2.split(' ')
    for c in b:
        if '#' in c:
            c2 = c.replace('#','')
            unique_text_hashtags.add(c2.lower())

In [29]:
print(len(unique_text_hashtags))
print(unique_text_hashtags)

77355
{'', 'emplaw', 'metastaticmisinformation', 'aajtak', 'arifwazir.', 'girlmedtwitter', 'southafrica’s', 'rss', 'dpr', 'learningfromhome', 'incarcerated.', 'endmassincarceration', 'boycottusmeat', 'todaijitemple', 'dingleybearsadventures', 'traumateam', 'fakemedia', 'ywomencount', 'திமுகதலைவர்', 'needvolunteers', 'rethinkingdemocracy', 'policybriefs,', 'acvc_esc', 'covresearch', 'reseach', 'america⁉️', 'jagadisiplin', '💉💯worldimmunizationweek', 'fairfieldcounty', 'experiment', 'suriyafansclub', 'ministryofhealth', 'bolsonaroheroi', 'researchbybswn', 'demo2', 'pandemics,incl.covid19', 'letsbeatcovid19together', 'icke', 'elpaso.', 'cholesterol,', 'cargileforcongress', 'innio', 'px2020', 'physicianburnout', 'covid19help', 'climatecrisis', 'day32oflockdown', '\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063yemen', 'digped', 'bellanaijaweddjngs', 'taiwanisacountry', '4thofmay?', 'yorkuscs', 'lambily

In [30]:
total_hashtags = unique_hashtags.union(unique_text_hashtags)

In [31]:
print(len(total_hashtags))
print(total_hashtags)

79411
{'emplaw', '', 'metastaticmisinformation', 'aajtak', 'dpr', 'boycottusmeat', 'ywomencount', 'திமுகதலைவர்', 'policybriefs,', 'rethinkingdemocracy', 'acvc_esc', 'reseach', 'america⁉️', '💉💯worldimmunizationweek', 'experiment', 'demo2', 'elpaso.', 'cholesterol,', 'cargileforcongress', 'innio', 'px2020', 'physicianburnout', 'covid19help', '\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063yemen', 'digped', 'bellanaijaweddjngs', '4thofmay?', 'yorkuscs', 'magnainaction', 'office,', 'celebratevaccines', 'weareallin.', 'taxheaven', 'lengthsofimmunity', 'sharingpractice', 'broadcasting', 'medicalacupuncture', 'changethestory', 'flue,', 'smartcontracts', 'it', 'givemom', 'nfid', 'riverthames', 'frontlinehealthworkers', 'pharmacyhour', 'firstdaynerves', 'copper.', 'reviewer', 'ppeshortage', 'jetstreamfest', 'adbnews', 'ouija', 'coop', 'infectiousdisease.', 'sakagibi', 'meetings.we4', 'wifi', 'chenqiushi,'

In [34]:
modified.insert(0, 'covid', modified.text.apply(lambda x: 1 if "covid" in x.lower() else 0))

In [37]:
modified.text[modified.covid!=0]

6         Extremely valid points being made here 👇🏾 #Eth...
7         COVID-19 dominated the discussion Tuesday at a...
8         BC now has 112 patients on ventilators.  17 of...
9                a COVID-19 vaccine would be pretty lit ngl
10        Man Caught With Pipe Bombs Was Planning Armed ...
                                ...                        
665753    Making remarks at the Virtual Online High-leve...
665759    can detect covid in sewers\nup to 6 days befor...
665771    Was supposed to be on my way to long Island NY...
665773    Thank you to all of the nurses in our @Stanfor...
665774    'Post it' pearls for Palliative, End of Life a...
Name: text, Length: 164321, dtype: object

In [38]:

def insert_bool_exists(word):
    modified.insert(0, word, modified.text.apply(lambda x: 1 if word in x.lower() else 0))

In [39]:
insert_bool_exists('trump')
insert_bool_exists('corona')
insert_bool_exists('virus')

In [41]:
modified.text[modified.trump!=0]

2         Old dirty tricks Trump, at it again...like we ...
49        Welcome to your brain on Trump! https://t.co/q...
71        Covid-19 deaths climbing &amp; Trump wants sta...
72        Trump pardoned war criminals, but Reality Winn...
189       I would hold China responsible for the Coronav...
                                ...                        
665586    NEW SINGLE OUT NOW MARATHON\n\nCLICK LINK BELO...
665612                                    Trump Lies again.
665645                           Trumps Big Lie of the Day!
665689    Amy McGrath is going to bury this Trump kissas...
665702    Trump will institute martial law one way or an...
Name: text, Length: 24706, dtype: object

In [42]:
modified.text[modified.corona!=0]

13        Why does Superstitious times Stupid always equ...
16        NEW SOCIAL DISTANCING HANDSHAKES SURGE IN THE ...
23        We’re Calling FOUL PLAY:\nNancy Pelosi Appoint...
26        Coronavirus. La quasi-totalité des marins du C...
31        Have you done any of these yet? #quarantine #c...
                                ...                        
665756    0.0483674% of the world has reportedly had Cor...
665757    YS Jagan to launch first made in Andhra Prades...
665762    Governor orders all roads closed into New Mexi...
665765    A great video from #Singapore raising awarenes...
665770    The NHS was protected from overload by forcing...
Name: text, Length: 113046, dtype: object

In [43]:
modified.text[modified.virus!=0]

13        Why does Superstitious times Stupid always equ...
16        NEW SOCIAL DISTANCING HANDSHAKES SURGE IN THE ...
23        We’re Calling FOUL PLAY:\nNancy Pelosi Appoint...
26        Coronavirus. La quasi-totalité des marins du C...
31        Have you done any of these yet? #quarantine #c...
                                ...                        
665756    0.0483674% of the world has reportedly had Cor...
665757    YS Jagan to launch first made in Andhra Prades...
665762    Governor orders all roads closed into New Mexi...
665765    A great video from #Singapore raising awarenes...
665770    The NHS was protected from overload by forcing...
Name: text, Length: 118621, dtype: object

In [45]:
print("virus")
print((modified.retweet_count[modified.virus!=0]).describe())
print()

print("corona")
print((modified.retweet_count[modified.corona!=0]).describe())
print()

print("trump")
print((modified.retweet_count[modified.trump!=0]).describe())
print()

print("covid")
print((modified.retweet_count[modified.covid!=0]).describe())
print()

virus
count    118621.000000
mean        299.318122
std        3505.800463
min           0.000000
25%           0.000000
50%           1.000000
75%          13.000000
max      241820.000000
Name: retweet_count, dtype: float64

corona
count    113046.000000
mean        303.879138
std        3534.755075
min           0.000000
25%           0.000000
50%           2.000000
75%          14.000000
max      241820.000000
Name: retweet_count, dtype: float64

trump
count     24706.000000
mean        398.343358
std        2850.468724
min           0.000000
25%           0.000000
50%           1.000000
75%          20.000000
max      145836.000000
Name: retweet_count, dtype: float64

covid
count    164321.000000
mean        138.749892
std        2176.856914
min           0.000000
25%           0.000000
50%           2.000000
75%          10.000000
max      235681.000000
Name: retweet_count, dtype: float64



In [47]:
column_1 = modified["corona"]
column_2 = modified["virus"]
correlation = column_1.corr(column_2)
print(correlation)

0.9227101893024494


In [48]:
column_1 = modified["trump"]
column_2 = modified["virus"]
correlation = column_1.corr(column_2)
print(correlation)

0.07282701047687168


In [49]:
column_1 = modified["covid"]
column_2 = modified["virus"]
correlation = column_1.corr(column_2)
print(correlation)

-0.06576643817699505


In [50]:
column_1 = modified["covid"]
column_2 = modified["corona"]
correlation = column_1.corr(column_2)
print(correlation)

-0.06732517534711809


In [ ]:
#drop corona

In [51]:
insert_bool_exists('usa')
insert_bool_exists('us')
insert_bool_exists('america')

In [54]:
print("usa")
print((modified.retweet_count[modified.usa!=0]).describe())
print()
print("us")
print((modified.retweet_count[modified.us!=0]).describe())
print()
print("america")
print((modified.retweet_count[modified.america!=0]).describe())


usa
count     7007.000000
mean       208.822035
std       2055.959194
min          0.000000
25%          0.000000
50%          2.000000
75%         17.000000
max      58281.000000
Name: retweet_count, dtype: float64

us
count    238212.000000
mean        230.267678
std        2924.357102
min           0.000000
25%           0.000000
50%           1.000000
75%           9.000000
max      241820.000000
Name: retweet_count, dtype: float64

america
count     14306.000000
mean        496.399692
std        4924.056299
min           0.000000
25%           0.000000
50%           1.000000
75%          18.000000
max      241820.000000
Name: retweet_count, dtype: float64


In [ ]:
#add america 

In [57]:
column_1 = modified["us"]
column_2 = modified["usa"]
correlation = column_1.corr(column_2)
print(correlation)

0.13817147949124384


KeyError: False

In [60]:
insert_bool_exists('support')
print("support")
print((modified.retweet_count[modified.support!=0]).describe())

support
count     13221.000000
mean        130.565615
std        1703.920140
min           0.000000
25%           1.000000
50%           3.000000
75%          14.000000
max      133198.000000
Name: retweet_count, dtype: float64


In [61]:
insert_bool_exists('help')
print("help")
print((modified.retweet_count[modified.help!=0]).describe())

help
count     19511.000000
mean        180.754497
std        2412.510025
min           0.000000
25%           1.000000
50%           3.000000
75%          14.000000
max      208130.000000
Name: retweet_count, dtype: float64


In [62]:
insert_bool_exists('doctor')
print("doctor")
print((modified.retweet_count[modified.doctor!=0]).describe())

doctor
count      5267.000000
mean        477.842605
std        4390.574834
min           0.000000
25%           0.000000
50%           3.000000
75%          29.000000
max      156817.000000
Name: retweet_count, dtype: float64


In [65]:
insert_bool_exists('nurse')
print("nurse")
print((modified.retweet_count[modified.nurse!=0]).describe())

nurse
count      2363.000000
mean        454.400762
std        4558.876757
min           0.000000
25%           1.000000
50%           5.000000
75%          36.000000
max      156817.000000
Name: retweet_count, dtype: float64


In [67]:
insert_bool_exists('vaccin')
print("vaccin")
print((modified.retweet_count[modified.vaccin!=0]).describe())

vaccin
count      7252.000000
mean        154.877551
std        2248.102363
min           0.000000
25%           0.000000
50%           1.000000
75%          11.000000
max      123334.000000
Name: retweet_count, dtype: float64


In [68]:
insert_bool_exists('lockdown')
print("lockdown")
print((modified.retweet_count[modified.lockdown!=0]).describe())

lockdown
count    14888.000000
mean       105.674033
std        899.840475
min          0.000000
25%          0.000000
50%          3.000000
75%         16.000000
max      42806.000000
Name: retweet_count, dtype: float64


In [69]:
insert_bool_exists('quarantine')
print("quarantine")
print((modified.retweet_count[modified.quarantine!=0]).describe())

quarantine
count      4035.000000
mean        377.439653
std        3975.007013
min           0.000000
25%           0.000000
50%           2.000000
75%          20.000000
max      107476.000000
Name: retweet_count, dtype: float64


In [70]:
insert_bool_exists('stay')
print("stay")
print((modified.retweet_count[modified.stay!=0]).describe())

stay
count     16393.000000
mean        198.767035
std        2535.685885
min           0.000000
25%           0.000000
50%           2.000000
75%          11.000000
max      130964.000000
Name: retweet_count, dtype: float64


In [71]:
insert_bool_exists('econom')
print("econom")
print((modified.retweet_count[modified.econom!=0]).describe())

econom
count     8516.000000
mean       161.449037
std       1745.114872
min          0.000000
25%          1.000000
50%          3.000000
75%         19.000000
max      89282.000000
Name: retweet_count, dtype: float64


In [72]:
insert_bool_exists('home')
print("home")
print((modified.retweet_count[modified.home!=0]).describe())

home
count     21390.000000
mean        209.558532
std        2531.252634
min           0.000000
25%           0.000000
50%           2.000000
75%          14.000000
max      130964.000000
Name: retweet_count, dtype: float64


In [73]:
insert_bool_exists('hero')
print("hero")
print((modified.retweet_count[modified.hero!=0]).describe())

hero
count     3248.000000
mean       173.601909
std       1809.776558
min          0.000000
25%          1.000000
50%          3.000000
75%         13.000000
max      67659.000000
Name: retweet_count, dtype: float64


In [74]:
insert_bool_exists('crisis')
print("crisis")
print((modified.retweet_count[modified.crisis!=0]).describe())

crisis
count     10486.000000
mean        207.644478
std        2283.052632
min           0.000000
25%           1.000000
50%           4.000000
75%          21.000000
max      104005.000000
Name: retweet_count, dtype: float64


In [75]:
insert_bool_exists('pray')
print("pray")
print((modified.retweet_count[modified.pray!=0]).describe())

pray
count     3546.000000
mean       185.144388
std       1838.039323
min          0.000000
25%          0.000000
50%          0.000000
75%          4.000000
max      50838.000000
Name: retweet_count, dtype: float64


In [76]:
insert_bool_exists('god')
print("god")
print((modified.retweet_count[modified.god!=0]).describe())

god
count      7692.000000
mean        145.824883
std        3085.202982
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max      223983.000000
Name: retweet_count, dtype: float64


In [80]:
def try_word(word):
    insert_bool_exists(word)
    print(word)
    print((modified.retweet_count[modified[word]!=0]).describe())
    print()


In [81]:
try_word("remote")


ValueError: cannot insert remote, already exists

In [83]:
print((modified.retweet_count[modified["remote"]!=0]).describe())

count     1032.000000
mean       263.590116
std       3985.563465
min          0.000000
25%          1.000000
50%          2.000000
75%          9.000000
max      94653.000000
Name: retweet_count, dtype: float64


In [84]:
try_word("zoom")
try_word("online")

zoom
count      747.000000
mean       139.485944
std       1471.490420
min          0.000000
25%          0.000000
50%          1.000000
75%          5.000000
max      32133.000000
Name: retweet_count, dtype: float64
online
count     4027.000000
mean       113.622299
std       1553.035469
min          0.000000
25%          1.000000
50%          3.000000
75%         11.000000
max      59596.000000
Name: retweet_count, dtype: float64


In [85]:
try_word("mask")

mask
count      8149.000000
mean        252.535035
std        3198.895883
min           0.000000
25%           0.000000
50%           1.000000
75%          13.000000
max      159168.000000
Name: retweet_count, dtype: float64


In [86]:
try_word("case")

case
count    19232.000000
mean       151.827943
std       1797.780814
min          0.000000
25%          0.000000
50%          1.000000
75%          7.000000
max      89282.000000
Name: retweet_count, dtype: float64


In [87]:
try_word("positive")

positive
count     6867.000000
mean       194.147663
std       2057.712731
min          0.000000
25%          0.000000
50%          1.000000
75%         10.000000
max      88682.000000
Name: retweet_count, dtype: float64


In [88]:
try_word("test")

test
count     29673.000000
mean        237.065919
std        2989.605502
min           0.000000
25%           0.000000
50%           1.000000
75%          12.000000
max      241820.000000
Name: retweet_count, dtype: float64


In [89]:
try_word("social")
try_word("government")

social
count      8753.000000
mean        308.464298
std        5755.400399
min           0.000000
25%           1.000000
50%           3.000000
75%          17.000000
max      415787.000000
Name: retweet_count, dtype: float64
government
count     12152.000000
mean        224.472021
std        2929.069978
min           0.000000
25%           0.000000
50%           2.000000
75%          17.000000
max      208844.000000
Name: retweet_count, dtype: float64


In [90]:
try_word("diag")

diag
count     1125.000000
mean       275.664889
std       2938.598844
min          0.000000
25%          1.000000
50%          4.000000
75%         28.000000
max      89285.000000
Name: retweet_count, dtype: float64


In [91]:
try_word("medicine")

medicine
count    1692.000000
mean       99.501182
std       613.828801
min         0.000000
25%         1.000000
50%         3.000000
75%        14.000000
max      9635.000000
Name: retweet_count, dtype: float64


In [92]:
try_word("2020")

2020
count     10843.000000
mean        156.365489
std        2172.550142
min           0.000000
25%           0.000000
50%           2.000000
75%          10.000000
max      124257.000000
Name: retweet_count, dtype: float64


In [93]:
try_word("\n\n")




count     79969.000000
mean        317.945041
std        3178.380602
min           0.000000
25%           1.000000
50%           5.000000
75%          34.000000
max      256197.000000
Name: retweet_count, dtype: float64


In [95]:
try_word(";")

;
count     34466.000000
mean        252.879650
std        4192.920281
min           0.000000
25%           1.000000
50%           4.000000
75%          22.000000
max      647993.000000
Name: retweet_count, dtype: float64
